# Waterfilling approximation using GNN 

In this notebook, we are going to examine a simple application of GNNs in wireless communications which is function approximation. Typically, the problem of power distribution among a set of users can be defined as: 


\begin{align}
 \max_{p_u} &\sum_u w_u \log(1+p_u G_u/\sigma^2) \\
 subject ~to:& \sum_u p_u \leq P 
\end{align}


where, P is the total power budget, $\sigma^2$ is the noise power ,$G_u$ and $w_u$ is the channel gain  and weight of  user $u$ , respectively. 

The solution of the above problem is simply given by the well-known waterfilling algorithm, derived from the Lagrangian of the above problem. The power per user is defined as:

$p_u = [\frac{w_u}{\mu} - \frac{1}{G_u}]^+$,
where $[.]^+ = max(0,.)$ Thus, we only need to solve the non-linear equation:

\begin{equation}
 \sum_u [\frac{w_u}{\mu} - \frac{1}{G_u}]^+ = P,
\end{equation}

which is done simply using a bisection method, or a backtracking method.


In order to approximate the waterfilling algorithm with a GNN, we will implement the following steps: 

1. Define the waterfilling function 
2. Use the function to generate a dataset ( a list of graphs) 
3. Create a dataloader, Define the model and training parameters
4. Train the model 

## Step 1: Waterfilling function implemenation 

For simplicity, we will normalize the variables $p_u$ by premultiplying the channel gains $G_u$ by $P/\sigma^2$ in the below implemenation. We will use torch for all numerical operations.

In [22]:
import torch 

def waterfilling_backtracking(user_weights,channel_gains):
    num_users = len(user_weights)
    I_u = torch.ones(num_users) # indicator of which users has a non-zero power
    total_power = 1 
    while torch.sum(I_u) > 0 : 
       inv_mu = (total_power + torch.sum(I_u * 1/channel_gains))/torch.sum(I_u * user_weights)
       allocated_power = I_u * (user_weights*inv_mu - 1/channel_gains)
       # determine set of users assigned a negative rate
       Sinactive = torch.where(allocated_power < 0 )[0]
       if len(Sinactive) == 0 :
           break
       I_u[Sinactive] = 0 
    return allocated_power



def waterfilling_bisection(user_weights,channel_gains,epsilon = 1e-6):
    total_power = 1 
    mu_min = 0.0  # this waterlevel results in a feasible allocation, yet sub-optimal 
    mu_max = (total_power + torch.sum(1/channel_gains))/torch.sum(user_weights) + 0.1 # this infeasible waterlevel
    num_users = len(user_weights) 
    
    while mu_max- mu_min > epsilon:
        mu = (mu_min+mu_max) / 2
        allocated_power = torch.maximum(torch.zeros(num_users),user_weights* mu - 1/channel_gains)
        power_sum = torch.sum(allocated_power)

        if power_sum <= total_power:
            mu_min = mu 
        else: 
            mu_max = mu
        
        return allocated_power


## Step 2: Generate a dataset of sample realization and the corresponding optimal solution 

We will use a simplified channel model as the goal of this exercise is to present the feasibiltiy of this approach. We define below a function to generate a single sample.

In [23]:
from torch_geometric.data import Data 

def generate_sample(_):
    min_users = 5 
    max_users = 70 
    
    total_power = 40 
    noise_power = 1e-5

    num_users = torch.randint(min_users,max_users+1,size=()) 
    user_weights = torch.rand(num_users)*0.9 + 0.1 
    channel_gains = torch.empty(num_users).exponential_(1.0)
    
    channel_gains = channel_gains * total_power/noise_power
    
    allocated_power  = waterfilling_backtracking(user_weights,channel_gains)
    

    edge_index = torch.tensor([(i,j) for i in range(num_users) for j in range(num_users) if i != j],dtype= torch.long).T
    
    data_graph = Data(
        num_nodes = num_users,
        w = user_weights,
        g = channel_gains,
        p = allocated_power ,
        edge_index = edge_index
          )
    data_graph.validate()
    return data_graph

num_samples = 10000

graph_dataset = [ ]
for i in range(num_samples):
    graph_dataset.append(generate_sample(_))

## Step 3: prepare for training 

First, we create a dataloader to pass data one graph at a time. 

In [24]:
from torch_geometric.loader import DataLoader

batch_size = 1
my_dataloader = DataLoader(graph_dataset,batch_size=batch_size,shuffle=True)

Second, we define the model to be used. We will use a graph convolutional neural network for its simplicty and since it will support any number of users as with the original algorithm. 

In [25]:
import torch.nn as nn 
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCNModel(nn.Module):
    def __init__(self,in_channels,hidden_channels,out_channels):
        super(GCNModel,self).__init__()
        self.conv1 = GCNConv(in_channels,hidden_channels)
        self.conv2 = GCNConv(hidden_channels,out_channels)
    def forward(self,x1,x2,edge_index):
        x = torch.stack((x1,x2),dim = -1)
        x = self.conv1(x,edge_index)
        x = F.relu(x)
        x = self.conv2(x,edge_index)
        x =  F.softmax(x,dim=0)
        
        return x 
    

Initialize the model, specify the loss function and the optimizer

In [43]:
model = GCNModel(in_channels=2,hidden_channels=16,out_channels=1)

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(),lr = 0.0001)

num_epochs = 5

## Step 4: training loop

In order to assess the quality of the model pre-training, we will generate a single batch, pass it trough the model and get the output weighted sum-rate. 

In [27]:
def compute_weighted_rate(channel_gains, user_weights,allocated_power):
    SNR = channel_gains* allocated_power
    #print(f'SNR shape is: {SNR.shape}')
    weighted_rate = user_weights * torch.log2(1+SNR)
    return torch.sum(weighted_rate)


batch = next(iter(my_dataloader))
print(batch)
output = model(batch.w,batch.g,batch.edge_index)



DataBatch(edge_index=[2, 1190], num_nodes=35, w=[35], g=[35], p=[35], batch=[35], ptr=[2])


In [28]:
wsr_ideal = compute_weighted_rate(channel_gains =batch.g,
                      user_weights = batch.w,
                      allocated_power = batch.p)
wsr_model = compute_weighted_rate(channel_gains =batch.g,
                      user_weights = batch.w,
                      allocated_power = torch.squeeze(output),
                      )

print(f' WSR with ML is {wsr_model.item():.4f} and the ideal is {wsr_ideal:.4f} ')

 WSR with ML is 325.9265 and the ideal is 329.4325 


Define the training loop using supervised learning. 

In [29]:
for epoch in range(num_epochs):
    model.train()

    total_loss = 0 
    total_wsr_ideal = 0 
    total_wsr_ML = 0

    for batch in my_dataloader: 
        output = model(batch.w,batch.g,batch.edge_index)

        loss = criterion(output,torch.reshape(batch.p,output.shape))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        wsr_ideal = compute_weighted_rate(channel_gains =batch.g,
                      user_weights = batch.w,
                      allocated_power = batch.p)
        wsr_model = compute_weighted_rate(channel_gains =torch.reshape(batch.g,output.shape),
                      user_weights = torch.reshape(batch.w,output.shape),
                      allocated_power = output,
                      )
        
        total_loss += loss.item()
        total_wsr_ideal += wsr_ideal
        total_wsr_ML += wsr_model.item()

    avg_loss = total_loss/len(my_dataloader)
    avg_wsr_ideal = total_wsr_ideal/len(my_dataloader)
    avg_wsr_ML = total_wsr_ML/len(my_dataloader)

    print(f'Epoch {epoch + 1}/{num_epochs}, loss: {avg_loss:.4f}, ideal WSR {avg_wsr_ideal:.4f}, avg_wsr_ML {avg_wsr_ML:.4f}, optimality gap is: {(avg_wsr_ideal-avg_wsr_ML)/avg_wsr_ideal*100:.4f} %')


    


Epoch 1/5, loss: 0.0007, ideal WSR 325.8770, avg_wsr_ML 322.3577, optimality gap is: 1.0799 %
Epoch 2/5, loss: 0.0007, ideal WSR 325.8767, avg_wsr_ML 322.3663, optimality gap is: 1.0772 %
Epoch 3/5, loss: 0.0007, ideal WSR 325.8760, avg_wsr_ML 322.3684, optimality gap is: 1.0764 %
Epoch 4/5, loss: 0.0007, ideal WSR 325.8761, avg_wsr_ML 322.3569, optimality gap is: 1.0799 %
Epoch 5/5, loss: 0.0007, ideal WSR 325.8768, avg_wsr_ML 322.3648, optimality gap is: 1.0777 %


Alternatively, we could use self-supervised learning, i.e., learning form the weighted sum-rate instead of learning form the MMSE criterion with the actual output. 

In [44]:
for epoch in range(num_epochs):
    model.train()

    total_loss = 0 
    total_wsr_ideal = 0 
    total_wsr_ML = 0

    for batch in my_dataloader: 
        output = model(batch.w,batch.g,batch.edge_index)

        wsr_ideal = compute_weighted_rate(channel_gains =batch.g,
                      user_weights = batch.w,
                      allocated_power = batch.p)
        wsr_model = compute_weighted_rate(channel_gains =torch.reshape(batch.g,output.shape),
                      user_weights = torch.reshape(batch.w,output.shape),
                      allocated_power = output,
                      )
        
        loss = 1 - wsr_model/wsr_ideal 

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        
        total_loss += loss.item()
        total_wsr_ideal += wsr_ideal
        total_wsr_ML += wsr_model.item()

    avg_loss = total_loss/len(my_dataloader)
    avg_wsr_ideal = total_wsr_ideal/len(my_dataloader)
    avg_wsr_ML = total_wsr_ML/len(my_dataloader)

    print(f'Epoch {epoch + 1}/{num_epochs}, loss: {avg_loss:.4f}, ideal WSR {avg_wsr_ideal:.4f}, avg_wsr_ML {avg_wsr_ML:.4f}, optimality gap is: {(avg_wsr_ideal-avg_wsr_ML)/avg_wsr_ideal*100:.4f} %')

Epoch 1/5, loss: 0.0105, ideal WSR 325.8770, avg_wsr_ML 322.3629, optimality gap is: 1.0784 %
Epoch 2/5, loss: 0.0105, ideal WSR 325.8763, avg_wsr_ML 322.3602, optimality gap is: 1.0790 %
Epoch 3/5, loss: 0.0105, ideal WSR 325.8756, avg_wsr_ML 322.3607, optimality gap is: 1.0786 %
Epoch 4/5, loss: 0.0105, ideal WSR 325.8770, avg_wsr_ML 322.3595, optimality gap is: 1.0794 %
Epoch 5/5, loss: 0.0105, ideal WSR 325.8767, avg_wsr_ML 322.3603, optimality gap is: 1.0791 %
